In [1]:
# Loading packages

import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
# Load the DataFrame from the pickle file
df = pd.read_pickle('documents_topics.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57790 entries, 0 to 57789
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      57790 non-null  object 
 1   text                          57790 non-null  object 
 2   created_at                    57790 non-null  object 
 3   campaign_week                 57790 non-null  int64  
 4   process_text_check            57790 non-null  object 
 5   name                          57790 non-null  object 
 6   handle                        57790 non-null  object 
 7   party                         57790 non-null  object 
 8   state_code                    57790 non-null  object 
 9   state_name                    57790 non-null  object 
 10  result_pctg                   57790 non-null  float64
 11  result_votes                  57790 non-null  int64  
 12  position                      57790 non-null  int64  
 13  t

## Test for H1a and H1b

In [3]:
df_h1a_h1b = df

def is_empty_or_nan(x):
    if x != x or not x:
        return True
    return False

# Create 'local' and 'national' columns
df_h1a_h1b['local'] = df_h1a_h1b['check_city_filtered'].apply(lambda x: 1 if not is_empty_or_nan(x) else 0)
df_h1a_h1b['national'] = df_h1a_h1b['check_city_filtered'].apply(lambda x: 1 if is_empty_or_nan(x) else 0)

In [4]:
df_h1a_h1b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57790 entries, 0 to 57789
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      57790 non-null  object 
 1   text                          57790 non-null  object 
 2   created_at                    57790 non-null  object 
 3   campaign_week                 57790 non-null  int64  
 4   process_text_check            57790 non-null  object 
 5   name                          57790 non-null  object 
 6   handle                        57790 non-null  object 
 7   party                         57790 non-null  object 
 8   state_code                    57790 non-null  object 
 9   state_name                    57790 non-null  object 
 10  result_pctg                   57790 non-null  float64
 11  result_votes                  57790 non-null  int64  
 12  position                      57790 non-null  int64  
 13  t

In [5]:
#remove unnecessary columns for a linear regression and statistical tests
df_h1a_h1b = df_h1a_h1b.drop(columns=['tweet_id','text','created_at','campaign_week','process_text_check','handle','party','state_code','state_name','total_votes_casted','check_city_filtered','PreProcessedText_WithoutCity','Words','WordsCleaned','WordsLemmatized','Campaign Event','Social Protection','General Discussion','Abortion','Homeland Security','Candidate Denigration','Vote Instigation','Campaign Contribution','Economy Policies','result_votes'], axis=1)
df_h1a_h1b.head(10)

,name,result_pctg,position,winner_margin_for_runners_up,local,national
0,Don Bolduc,0.444346,2,0.090608,1,0
1,Don Bolduc,0.444346,2,0.090608,1,0
2,Don Bolduc,0.444346,2,0.090608,1,0
3,Don Bolduc,0.444346,2,0.090608,0,1
4,Don Bolduc,0.444346,2,0.090608,0,1
5,Don Bolduc,0.444346,2,0.090608,0,1
6,Don Bolduc,0.444346,2,0.090608,1,0
7,Don Bolduc,0.444346,2,0.090608,0,1
8,Don Bolduc,0.444346,2,0.090608,1,0
9,Don Bolduc,0.444346,2,0.090608,1,0


In [6]:
#aggregate by name
#create a dataframe with the values for the algorithms
#transform position into a categorical variable

df_h1a_h1b['position'] = df_h1a_h1b['position'].astype('category')


agg_dict = {
    #aggregate the numerical values
    'local':'sum',
    'national':'sum',
    #these values are equal for all the tweets from the same person
    'result_pctg':'mean',
    'winner_margin_for_runners_up':'mean',
    #now the categorical values
    'position':'last'
}


#create the new dataframe
df_h1a_h1b = df_h1a_h1b.groupby(['name']).agg(agg_dict).reset_index()
df_h1a_h1b.head(50)


,name,local,national,result_pctg,winner_margin_for_runners_up,position
0,Adam Laxalt,523,540,0.480372,0.007766,2
1,Alex Padilla,201,571,0.591357,0.182714,1
2,Bill Redpath,5,9,0.016754,0.153232,3
3,Blake Masters,75,501,0.465074,0.048874,2
4,Bob McDermott,1,4,0.227386,0.521716,2
5,Brian Bengs,211,510,0.250897,0.458107,2
6,Catherine Cortez Masto,448,683,0.488138,0.007766,1
7,Charles Booker,522,1009,0.382045,0.235819,2
8,Cheri Beasley,479,466,0.472731,0.032257,2
9,Chris Chaffee,111,675,0.340749,0.316932,2


In [7]:
df_h1a_h1b.head(100)

,name,local,national,result_pctg,winner_margin_for_runners_up,position
0,Adam Laxalt,523,540,0.480372,0.007766,2
1,Alex Padilla,201,571,0.591357,0.182714,1
2,Bill Redpath,5,9,0.016754,0.153232,3
3,Blake Masters,75,501,0.465074,0.048874,2
4,Bob McDermott,1,4,0.227386,0.521716,2
...,...,...,...,...,...,...
74,Tim Scott,46,782,0.620044,0.240089,1
75,Todd Young,178,257,0.586183,0.207461,1
76,Trudy Busch Valentine,316,343,0.421769,0.132554,2
77,Val Demings,337,610,0.412673,0.164130,2


In [8]:
#check for the proportion of tweets and normalize values for linear regression
df_h1a_h1b['national_proportion'] = df_h1a_h1b['national'] / (df_h1a_h1b['local'] + df_h1a_h1b['national'])
df_h1a_h1b['local_proportion'] = df_h1a_h1b['local'] / (df_h1a_h1b['national'] + df_h1a_h1b['local'])

#drop local and national columns
winners_df = df_h1a_h1b[df_h1a_h1b['position']==1]
winners_df = winners_df.drop(columns=['local','national','name','result_pctg','position'])
winners_df.head(10)

,winner_margin_for_runners_up,national_proportion,local_proportion
1,0.182714,0.739637,0.260363
6,0.007766,0.603890,0.396110
10,0.316932,0.863636,0.136364
11,0.128551,0.566197,0.433803
12,0.132112,0.687500,0.312500
15,0.132554,0.699495,0.300505
20,0.061119,0.784689,0.215311
22,0.322140,0.805195,0.194805
25,0.229624,0.660377,0.339623
29,0.348388,0.801619,0.198381


In [9]:
# Normalize the variables
winners_df['normalized_national_proportion'] = (winners_df['national_proportion'] - winners_df['national_proportion'].mean()) / winners_df['national_proportion'].std()
winners_df['normalized_local_proportion'] = (winners_df['local_proportion'] - winners_df['local_proportion'].mean()) / winners_df['local_proportion'].std()


#drop non-normalized values
winners_df = winners_df.drop(columns=['local_proportion','national_proportion'],axis=1)
winners_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 1 to 75
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   winner_margin_for_runners_up    34 non-null     float64
 1   normalized_national_proportion  34 non-null     float64
 2   normalized_local_proportion     34 non-null     float64
dtypes: float64(3)
memory usage: 1.1 KB


In [10]:
# Define a function to perform linear regression and print the results
def perform_linear_regression(data, dependent_var, independent_vars):
    Y = data[dependent_var]
    X = data[independent_vars]
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    print(model.summary())


# Perform linear regression with only winners using normalized variables for national_proportion
print("Winners with Normalized Variables - National Proportion:")
perform_linear_regression(winners_df, 'winner_margin_for_runners_up', ['normalized_national_proportion'])

# Perform linear regression with only winners using normalized variables for local_proportion
print("Winners with Normalized Variables - Local Proportion:")
perform_linear_regression(winners_df, 'winner_margin_for_runners_up', ['normalized_local_proportion'])


Winners with Normalized Variables - National Proportion:
                                 OLS Regression Results                                 
Dep. Variable:     winner_margin_for_runners_up   R-squared:                       0.101
Model:                                      OLS   Adj. R-squared:                  0.073
Method:                           Least Squares   F-statistic:                     3.597
Date:                          Tue, 09 May 2023   Prob (F-statistic):             0.0669
Time:                                  21:20:04   Log-Likelihood:                 24.090
No. Observations:                            34   AIC:                            -44.18
Df Residuals:                                32   BIC:                            -41.13
Df Model:                                     1                                         
Covariance Type:                      nonrobust                                         
                                     coef    std err 

/Users/franciscorfafonso/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
/Users/franciscorfafonso/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)





## Results




In [12]:
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

stargazer = Stargazer([national_results, local_results])
HTML(stargazer.render_html())# Add a constant to the DataFrame for the regression intercept
winners_df['const'] = 1

# Define the regression formula for local tweets
local_model = sm.OLS(winners_df['normalized_local_proportion'], winners_df[['const', 'winner_margin_for_runners_up']])

# Fit the model
local_results = local_model.fit()

# Print out the results
print(local_results.summary())

# Define the regression formula for national tweets
national_model = sm.OLS(winners_df['normalized_national_proportion'], winners_df[['const', 'winner_margin_for_runners_up']])

# Fit the model
national_results = national_model.fit()

# Print out the results
print(national_results.summary())

                                 OLS Regression Results                                
Dep. Variable:     normalized_local_proportion   R-squared:                       0.101
Model:                                     OLS   Adj. R-squared:                  0.073
Method:                          Least Squares   F-statistic:                     3.597
Date:                         Tue, 09 May 2023   Prob (F-statistic):             0.0669
Time:                                 21:42:07   Log-Likelihood:                -45.925
No. Observations:                           34   AIC:                             95.85
Df Residuals:                               32   BIC:                             98.90
Df Model:                                    1                                         
Covariance Type:                     nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
-----------------------

In [16]:
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

stargazer = Stargazer([national_results, local_results])
# Rename the models
stargazer.rename_covariates({'const': 'Intercept'})

# Change model names
stargazer.custom_columns(['National Tweets', 'Local Tweets'], [1, 1])
HTML(stargazer.render_html())

In [17]:
with open('regression_table_2.html', 'w') as f:
    f.write(stargazer.render_html())